In [2]:
# Import library for load data
import argparse
import ray
ray.init(num_cpus=20)
import modin.pandas as pd
#import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from category_encoders import OrdinalEncoder
from numpy import genfromtxt
import sys
import csv
# Import deeplearning library
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms 
from torch.utils.data import Dataset
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np 
from CustomFunction import *
from model_define import *

/.pyenv/versions/3.10.5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-26 14:14:13,588	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-07-26 14:14:16,693	INFO worker.py:1621 -- Started a local Ray instance.


In [4]:
WORK_PATH = os.getcwd()
validated_dir = WORK_PATH  + '/validated_balanced'
os.makedirs(validated_dir, exist_ok=True) 
#----------------PARSE PARAMETER-----------------

In [5]:
#----------------PARSE PARAMETER-----------------
parser = argparse.ArgumentParser(description='This program will take trained model and validate on dataset]')
# parser.add_argument: Add parameter to program
WORK_PATH = os.getcwd()
parser.add_argument('--device', type = int, help='Define GPU device for training', default=0) # Required parameter
parser.add_argument('--ver', help='Traning model version name (Scheduled, NonScheduled, Test) (default: Scheduled)', default='Scheduled') 
parser.add_argument('--dataset', help=f'Training dataset path (default: {WORK_PATH}/dataset/TrainCIC2019.csv)', default=WORK_PATH + '/dataset/TrainCIC2019.csv') # Required parameter
parser.add_argument('--model', help='Trained model name (ANN, ANND, DNN, DNND)(default: ANN)', default='ANN') 
parser.add_argument('--epoch', help='Trained model epoch_number (default: 100)', default='100') 
parser.add_argument('--bs', type = int, help='Batch size for traning (Default: 10000)', default= 10000)
parser.add_argument('--trained_type', help='Type of trained model ("train" for TrainCIC2019.csv or "test" for TestCIC2019.csv, default: train)', default='train') 
parser.add_argument('--test_type', help='Type of dataset ("train" for TrainCIC2019.csv or "test" for TestCIC2019.csv, default: train)', default='train') 
#args = parser.parse_args()
args = parser.parse_args(args=['--epoch','3', '--dataset', './dataset_ddos/CICIDS2017.csv', '--model', 'CNN', '--ver', '0', '--trained_type', 'CICIDS2017_anomaly', '--test_type', 'CICIDS2017_anomaly', '--bs', '1000'])

#----------------Define variable based of args----------
GPU_NUM = args.device
dataset_PATH = args.dataset
model_type = args.model
ver = args.ver
epoch_num = args.epoch
batch_size = args.bs
test_type = args.test_type
trained_type = args.trained_type
trained_model_PATH =  WORK_PATH + '/trained_model' + '/' + trained_type + '/' + model_type + '_' + ver + '/' + epoch_num + 'epoch.pth'
# Make folder for validate data file
print('-------------PARAMETERS-------------')
print('Current work path:',WORK_PATH)
print('Dataset PATH:', dataset_PATH)
print('Dataset type:', test_type)
print('Training on GPU:',GPU_NUM)
print('Batch size:', batch_size)
print('Trained model PATH:', trained_model_PATH)

# Define CUDA device 
device = torch.device(GPU_NUM if torch.cuda.is_available() else 'cpu')

-------------PARAMETERS-------------
Current work path: /home/CICIDS
Dataset PATH: ./dataset_ddos/CICIDS2017.csv
Dataset type: CICIDS2017_anomaly
Training on GPU: 0
Batch size: 1000
Trained model PATH: /home/CICIDS/trained_model/CICIDS2017_anomaly/CNN_0/3epoch.pth


In [7]:
print('-------------ANALYZE DATASET-------------')
analyze_start =  time.time()
#Load dataset
print("Loading dataset, please wait .....")
train = pd.read_csv(dataset_PATH)
#test = pd.read_csv('/home/jun/CICIDS2019/TestCIC2019.csv')
print("DONE!")
print("-------------------------")

# Fix columns name
print("Start fixing columns name")
train = fix_columns_name(train)
print("DONE!")
print("-------------------------")


# Drop NaN and inf
print("Start dropping NaN and Infinity contained row")
train = drop_NaN(train)
print("DONE!")
print("-------------------------")

# Encoding train dataset label
print("Start encoding flow labels to number")
train = encoding_labels(test_type, train)
print("DONE!")
print("-------------------------")
output_size = len(set(train['Label']))
# Drop num numberic columns ('Flow ID','Source IP','Destination IP', 'Timestamp', 'SimillarHTTP')
print("Dropping ('Unnamed: 0', 'Flow ID','Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'SimillarHTTP')")
try:
    train = drop_non_numberic_columns(train)
except:
    print("No columns to drop")
print("DONE!")
print("-------------------------")
input_size = len(train.columns)-1
output_size = len(set(train['Label']))
# Change data to numpy
print("Change data into numpy")
train = train.to_numpy()
train = np.asfarray(train)
print("DONE!")
print("-------------------------")

# Split dataset to data and label
print("Split numpy data to data and label")
print(train.shape)
print(input_size)
train_day_X = train[:, :input_size]
train_day_y = train[:, input_size]
print("DONE!")
print("-------------------------")

# Normalize data to range (0, 1)
print("Normalizing data into range [0,1] for training")
scaler = MinMaxScaler()
scaler.fit(train_day_X)
train_day_X = scaler.transform(train_day_X)
print("DONE!")
print("-------------------------")

# Slit training day data to train (90%), val (5%) and test (5%)
print("Split train, validate and test dataset: 90:5:5")
X_train, X_val_test, y_train, y_val_test = train_test_split(train_day_X, train_day_y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42) 
print("DONE!")
print('Train shape: {}, Validate shape: {}, Test shape: {}'.format(X_train.shape, X_val.shape, X_test.shape))
print("-------------------------")
analyze_end =  time.time()
print('Analyze data took: {:.2f}s'.format(analyze_end - analyze_start))

-------------ANALYZE DATASET-------------
Loading dataset, please wait .....
DONE!
-------------------------
Start fixing columns name
DONE!
-------------------------
Start dropping NaN and Infinity contained row
DONE!
-------------------------
Start encoding flow labels to number
DONE!
-------------------------
Dropping ('Unnamed: 0', 'Flow ID','Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'SimillarHTTP')
No columns to drop
DONE!
-------------------------
Change data into numpy
DONE!
-------------------------
Split numpy data to data and label
(1446598, 78)
77
DONE!
-------------------------
Normalizing data into range [0,1] for training
DONE!
-------------------------
Split train, validate and test dataset: 90:5:5
DONE!
Train shape: (1157278, 77), Validate shape: (144660, 77), Test shape: (144660, 77)
-------------------------
Analyze data took: 5.60s


In [11]:
test_dataset = np.concatenate((X_test,y_test.reshape(y_test.shape[0],1)),axis=1)

In [12]:
print(test_dataset.shape)

(144660, 78)


In [20]:

new_test_dataset = []
count_0 = 0
count_1 = 0
for flow in test_dataset:
    if flow[77] == 0 and count_0 < 5000:
        count_0 += 1
        new_test_dataset.append(flow) 
    if flow[77] == 1 and count_1 < 5000:
        count_1 += 1
        new_test_dataset.append(flow) 
new_test_dataset = np.array(new_test_dataset)

print('Benign:',count_0)
print('DDoS:',count_1)
print('new_test_dataset',new_test_dataset.shape)        

Benign: 5000
DDoS: 5000
new_test_dataset (10000, 78)


In [21]:
for flow in new_test_dataset:
    if flow[77] == 0:
        count_0 += 1
    if flow[77] == 1:
        count_1 += 1
print('Benign:',count_0)
print('DDoS:',count_1)

Benign: 10000
DDoS: 10000


In [22]:
X_test = new_test_dataset[:, :input_size]
y_test = new_test_dataset[:, input_size]

In [23]:
print('-------------TRAINING-------------')
Test_dataset = MyDataset(X_test.astype(np.float32), y_test.astype(np.float32))
# Create a data loader for batch training
test_loader = torch.utils.data.DataLoader(Test_dataset, batch_size=batch_size, shuffle=False)

-------------TRAINING-------------


In [24]:
# Create Model to load
# Define Model  
if model_type == 'ANN':
    model = ANN(input_size, output_size)
elif model_type == 'ANND':
    model = ANND(input_size, output_size)
elif model_type == 'DNN':
    model = DNN(input_size, output_size)
elif model_type == 'DNND':
    model = DNND(input_size, output_size)
elif model_type == 'CNN':
    model = CNN(input_size, output_size)
elif model_type == 'CNN2':
    model = CNN2(input_size, output_size)
elif model_type == 'CNN3':
    model = CNN3(input_size, output_size)

model.load_state_dict(torch.load(trained_model_PATH, map_location=device))
# Push model to GPU
model.to(device)
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
print(model)

CNN(
  (linear1): Linear(in_features=77, out_features=4800, bias=True)
  (conv1): Conv2d(3, 40, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(40, 200, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=9800, out_features=5000, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=5000, out_features=2, bias=True)
)


In [25]:
model.load_state_dict(torch.load(trained_model_PATH, map_location=device))
# Push model to GPU
model.to(device)
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
print(model)

def get_test_loss_acc(net, criterion, data_loader):
    """A simple function that iterates over `data_loader` to calculate the overall loss"""
    testing_loss = []
    testing_acc = []
    avg_loss = 0
    avg_acc = 0
    # Define confusion matrix
    c = torch.zeros((output_size, output_size))
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            # Get the data to GPU (if available)
            inputs, labels = inputs.to(device), labels.to(device, dtype = torch.long) # Change to data type long to not rise error
            outputs = net(inputs)
            # Calculate the loss for this batch
            loss = criterion(outputs, labels)
            # Add the loss of this batch to the list
            testing_loss.append(loss.item())
            _, predicted = torch.max(outputs, 1) # Get prediction on batch (shape: batch_size)
            # Add up confusion matrix in each batch to confusion matrix
            c += ConfusionMatrix(labels, predicted, output_size)
            batch_correct_predicted = (predicted == labels).sum().item() # Get number of true prediction on batch
            testing_acc.append(batch_correct_predicted/batch_size*100) # Append prediction accuracy of this batch
    # Calculate the average loss and accuracy
    avg_loss = np.mean(testing_loss) 
    avg_acc = np.mean(testing_acc) 
    return {'loss': avg_loss, 'acc': avg_acc, 'confusion_matrix':c}

CNN(
  (linear1): Linear(in_features=77, out_features=4800, bias=True)
  (conv1): Conv2d(3, 40, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(40, 200, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=9800, out_features=5000, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=5000, out_features=2, bias=True)
)


In [26]:
result = get_test_loss_acc(model, criterion, test_loader)


In [27]:

avg_test_loss = result['loss']
avg_test_acc = result['acc']
c_matrix = result['confusion_matrix'] 
validate_result = Validate(c_matrix)
actual = validate_result['actual']
total_acc = validate_result['accuracy'] 
precision = validate_result['precision'] 
precision_dict = {}
for i,p in enumerate(precision):
    precision_dict.update({i:np.around(float(p),decimals=3)})
    
recall = validate_result['recall'] 
recall_dict = {}
for i,r in enumerate(recall):
    recall_dict.update({i:np.around(float(r),decimals=3)})
f1_score = validate_result['f1_score'] 
f1_score_dict = {}
for i,f in enumerate(f1_score):
    f1_score_dict.update({i:np.around(float(f),decimals=3)})

In [28]:
txt_file_PATH = validated_dir + '/' + trained_type.split('_')[0] + '_' + test_type.split('_')[0] + '.txt' 
f = open(txt_file_PATH, 'w')

f.write('Test average accuracy: {:.2f}%, Test average loss: {:.3f}'.format(avg_test_acc, avg_test_loss))
f.write('\n')
print('Test average accuracy: {:.2f}%, Test average loss: {:.3f}'.format(avg_test_acc, avg_test_loss))
f.write(f'Confusion matrix:\n {c_matrix.type(torch.int).numpy()}')
f.write('\n')
print('Confusion matrix:\n', c_matrix.type(torch.int).numpy())
f.write(f'Benign : DDoS = {int(actual[0])} : {int(actual[1])} = {(actual[0]/torch.sum(actual)*100).item():.2f} : {(actual[1]/torch.sum(actual)*100).item():.2f}')
f.write('\n')
print(f'Benign : DDoS = {int(actual[0])} : {int(actual[1])} = {(actual[0]/torch.sum(actual)*100).item():.2f} : {(actual[1]/torch.sum(actual)*100).item():.2f}')
f.write(f'Total accuracy: {total_acc*100:.2f}%')
f.write('\n')
print(f'Total accuracy: {total_acc*100:.2f}%')
f.write(f'Precision: {precision_dict}')
f.write('\n')
print(f'Precision: {precision_dict}')
f.write(f'Recall: {recall_dict}')
f.write('\n')
print(f'Recall: {recall_dict}')
f.write(f'f1_score: {f1_score_dict}')
f.write('\n')
print(f'f1_score: {f1_score_dict}')
f.close()

Test average accuracy: 95.22%, Test average loss: 0.106
Confusion matrix:
 [[4895  105]
 [ 373 4627]]
Benign : DDoS = 5000 : 5000 = 50.00 : 50.00
Total accuracy: 95.22%
Precision: {0: 0.929, 1: 0.978}
Recall: {0: 0.979, 1: 0.925}
f1_score: {0: 0.953, 1: 0.951}
